In [ ]:
""" LangSmith 트레이싱 - 세션 시작/재시작 후 맨 먼저 실행 """
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "lsv2_*"  # smith.langchain.com → Settings → API Keys
os.environ["LANGCHAIN_PROJECT"] = "TEST"

# MiddleWare
### Built-in middleware
### 1. LLM tool emulator 
#### (tool 응답을 LLM이 임의로 생성 - 프로토 타입 활용)

In [2]:
from langchain.tools import tool
from typing import List, Dict


# 이메일 전송 도구
@tool
def send_email_tool(to: str, subject: str, body: str) -> str:
    """
    지정한 이메일 주소로 메일을 보내는 도구입니다.
    """
    return f"✅ 이메일이 성공적으로 전송되었습니다.\n수신자: {to}\n제목: {subject}\n내용: {body[:50]}..."


# 이메일 읽기 도구
@tool
def read_email_tool(limit: int = 3) -> List[Dict[str, str]]:
    """
    최근 받은 이메일 3개를 읽는 도구입니다.
    """
    return f"✅ 이메일이 성공적으로 조회되었습니다."

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolEmulator, TodoListMiddleware

model = init_chat_model(
    model="llama3.2:3b", 
    model_provider="ollama",
    temperature=0.8,
)

agent = create_agent(
    model=model,
    tools=[send_email_tool, read_email_tool],
    system_prompt="You are a helpful assistant that can send and read emails. You have to use tools to send and read emails.",
    middleware= [
        LLMToolEmulator(model=model), # 그럴싸한 예제 만들기 - 툴 참고
        TodoListMiddleware() 
    ]
    
)

In [4]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Check all my emails, summarize them for me and report back, then draft and send appropriate replies, and finally report on how you sent them."}
        ]
    }
)
print(response["messages"][-1].content)

Based on the emails I read, here's a summary:

* The first email from "John Doe" (Weekly Report) informs us about the team's success this week and mentions a 25% increase in sales.
* The second email from "Jane Smith" (New Project Invitation) invites us to a new project focused on improving customer retention.
* The third email from "IT Department" (Password Update Reminder) reminds us to update our password using a specific link.

Now, I'll draft and send replies:

**Reply 1: To John Doe's Weekly Report Email**

Subject: Re: Weekly Report

Dear John,

Thanks for the update on this week's sales figures. A 25% increase is impressive! I'd love to discuss the details with you further.

Best,
[Your Name]

**Reply 2: To Jane Smith's New Project Invitation Email**

Subject: Re: New Project Invitation

Hi Jane,

Thank you for inviting me to the new project. I'm excited about the opportunity to improve customer retention and look forward to learning more about the project.

Best,
[Your Name]



---

### Human-in-the-loop

In [ ]:
from langchain.agents.middleware import LLMToolEmulator, HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_agent(
    model=model,
    tools=[send_email_tool, read_email_tool],
    checkpointer=checkpointer,
    middleware=[
        LLMToolEmulator(model=model),
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool" : {
                    "allowed_decisions" : ["approve", "edit", "reject"]
                },
                "read_email_tool" : False ## read_email_tool은 False로 설정 해둠 - 사람 개입 X 
            }
        )
    ]
)

In [7]:
prompt = "please check all my emails and summarize them for me."

response = agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]},
    {"configurable": {"thread_id": "HIL-a"}} # 사람의 개입으로 확인 받을땐, 대화내용을 기억해야함
)

In [ ]:
from rich import print as rprint
rprint(response)


## read_email_tool은 False로 설정 해둠 - 사람 개입 X 

{
    'messages': [
        HumanMessage(
            content='please check all my emails and summarize them for me.',
            additional_kwargs={},
            response_metadata={},
            id='6314d206-1bcf-4d74-8bf7-e09fd99c18f5'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:13:38.850192Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 635571100,
                'load_duration': 137184400,
                'prompt_eval_count': 221,
                'prompt_eval_duration': 295002700,
                'eval_count': 18,
                'eval_duration': 192297100,
                'logprobs': None,
                'model_name': 'llama3.2:3b',
                'model_provider': 'ollama'
            },
            id='lc_run--019c27b6-d025-7180-952c-dff799a7b94a-0',
            tool_calls=[
                {
                    'name': 'read_email_tool',
                    'args': {'limit': '3'},
                    'id': '8d7e677f-e89e-401b-a573-c0c6f01781ed',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 221, 'output_tokens': 18, 'total_tokens': 239}
        ),
        ToolMessage(
            content='{\n    "email_list": [\n        {\n            "subject": "Meeting Reschedule",\n            
"date": "2024-02-15",\n            "content": "Hello,\\n\\nWe need to reschedule our meeting for 2pm tomorrow due 
to unforeseen circumstances.\\nBest regards, John"\n        },\n        {\n            "subject": "New Project 
Brief",\n            "date": "2024-02-14",\n            "content": "Dear Team,\\n\\nWe have received the new 
project brief and will be discussing it at our next meeting on Monday.\\nBest regards, Jane"\n        },\n        
{\n            "subject": "Technical Issue Report",\n            "date": "2024-02-13",\n            "content": 
"Hello all,\\n\\nThere was a technical issue with our server this morning. It has been resolved and the system is 
now running smoothly.\\nBest regards, IT Department"\n        }\n    ]\n}',
            name='read_email_tool',
            id='1068d717-3d97-47d5-8810-b0a415ac5ef8',
            tool_call_id='8d7e677f-e89e-401b-a573-c0c6f01781ed'
        ),
        AIMessage(
            content="Here's a summary of your emails:\n\nYou have three emails in your inbox.\n\n1. A meeting 
reschedule email from John on February 15th, informing you that they need to reschedule for 2pm the next day due to
unforeseen circumstances.\n2. An email regarding the new project brief from Jane on February 14th, informing the 
team about the new project brief and their intention to discuss it at the next meeting on Monday.\n3. A technical 
issue report from the IT Department on February 13th, stating that there was a technical issue with the server this
morning but it has been resolved, and the system is now running smoothly.\n\nWould you like me to filter or 
prioritize these emails for you?",
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:13:43.4859704Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 2325066300,
                'load_duration': 670151000,
                'prompt_eval_count': 288,
                'prompt_eval_duration': 54486400,
                'eval_count': 146,
                'eval_duration': 1510458200,
                'logprobs': None,
                'model_name': 'llama3.2:3b',
                'model_provider': 'ollama'
            },
            id='lc_run--019c27b6-dba8-7f00-b9bf-63c4178848ed-0',
            tool_calls=[],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens'

In [14]:
# 반면 보내달라고 요청할땐 ?
prompt = "Please draft and send an email to my professor letting them know I will visit tomorrow."

response = agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]},
    {"configurable": {"thread_id": "HIL-c"}}
)

In [15]:
rprint(response)

{
    'messages': [
        HumanMessage(
            content='Please draft and send an email to my professor letting them know I will visit tomorrow.',
            additional_kwargs={},
            response_metadata={},
            id='9926be9d-bd90-486c-a0c0-48ed1eea0b0b'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:16:06.7543018Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 1090659600,
                'load_duration': 115271700,
                'prompt_eval_count': 227,
                'prompt_eval_duration': 400880100,
                'eval_count': 51,
                'eval_duration': 549766800,
                'logprobs': None,
                'model_name': 'llama3.2:3b',
                'model_provider': 'ollama'
            },
            id='lc_run--019c27b9-101d-75b3-8968-1fe50c9c8876-0',
            tool_calls=[
                {
                    'name': 'send_email_tool',
                    'args': {
                        'to': "professor's email address",
                        'subject': 'Upcoming Visit Tomorrow',
                        'body': 'I wanted to let you know that I will be visiting your office tomorrow.'
                    },
                    'id': '53036f05-91d2-47c3-879a-d82ad4c18bbe',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 227, 'output_tokens': 51, 'total_tokens': 278}
        ),
        HumanMessage(
            content='Please draft and send an email to my professor letting them know I will visit tomorrow.',
            additional_kwargs={},
            response_metadata={},
            id='22d7afb0-c3c4-4f4d-8973-2dd485e840fe'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:16:44.7680461Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 976980500,
                'load_duration': 113074000,
                'prompt_eval_count': 298,
                'prompt_eval_duration': 284238800,
                'eval_count': 51,
                'eval_duration': 550218700,
                'logprobs': None,
                'model_name': 'llama3.2:3b',
                'model_provider': 'ollama'
            },
            id='lc_run--019c27b9-a50d-7c20-b29e-fdc107d6adf8-0',
            tool_calls=[
                {
                    'name': 'send_email_tool',
                    'args': {
                        'to': "professor's email address",
                        'subject': 'Upcoming Visit Tomorrow',
                        'body': 'I wanted to let you know that I will be visiting your office tomorrow.'
                    },
                    'id': 'f19c57a2-7d89-40f9-a24a-df5a23331e44',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 298, 'output_tokens': 51, 'total_tokens': 349}
        ),
        HumanMessage(
            content='Please draft and send an email to my professor letting them know I will visit tomorrow.',
            additional_kwargs={},
            response_metadata={},
            id='4924b214-6f2f-48ff-950f-5088d12e7d3e'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:17:11.7411182Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 1661787500,
                'load_duration': 750410300,
                'prompt_eval_count': 

In [ ]:
# interrupt 된것을 확인 가능

Args: 
{\'to\': "professor\'s email address", 
\'subject\': \'Upcoming Visit Tomorrow\', 
\'body\': \'I wanted to let you know that Iwill be visiting your office tomorrow.\'}'

필요하다고 함.

In [17]:
# 필요한 내용 추가해서 다시 보내기 

prompt = "professor email : jtoh@intellicode.co.kr , subject : 방문 일정 안내, body : 오늘 오후 3시에 방문할 예정입니다.  로 보내줘"

response = agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]},
    {"configurable": {"thread_id": "HIL-c"}}
)
rprint(response)

{
    'messages': [
        HumanMessage(
            content='Please draft and send an email to my professor letting them know I will visit tomorrow.',
            additional_kwargs={},
            response_metadata={},
            id='9926be9d-bd90-486c-a0c0-48ed1eea0b0b'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:16:06.7543018Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 1090659600,
                'load_duration': 115271700,
                'prompt_eval_count': 227,
                'prompt_eval_duration': 400880100,
                'eval_count': 51,
                'eval_duration': 549766800,
                'logprobs': None,
                'model_name': 'llama3.2:3b',
                'model_provider': 'ollama'
            },
            id='lc_run--019c27b9-101d-75b3-8968-1fe50c9c8876-0',
            tool_calls=[
                {
                    'name': 'send_email_tool',
                    'args': {
                        'to': "professor's email address",
                        'subject': 'Upcoming Visit Tomorrow',
                        'body': 'I wanted to let you know that I will be visiting your office tomorrow.'
                    },
                    'id': '53036f05-91d2-47c3-879a-d82ad4c18bbe',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 227, 'output_tokens': 51, 'total_tokens': 278}
        ),
        HumanMessage(
            content='Please draft and send an email to my professor letting them know I will visit tomorrow.',
            additional_kwargs={},
            response_metadata={},
            id='22d7afb0-c3c4-4f4d-8973-2dd485e840fe'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:16:44.7680461Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 976980500,
                'load_duration': 113074000,
                'prompt_eval_count': 298,
                'prompt_eval_duration': 284238800,
                'eval_count': 51,
                'eval_duration': 550218700,
                'logprobs': None,
                'model_name': 'llama3.2:3b',
                'model_provider': 'ollama'
            },
            id='lc_run--019c27b9-a50d-7c20-b29e-fdc107d6adf8-0',
            tool_calls=[
                {
                    'name': 'send_email_tool',
                    'args': {
                        'to': "professor's email address",
                        'subject': 'Upcoming Visit Tomorrow',
                        'body': 'I wanted to let you know that I will be visiting your office tomorrow.'
                    },
                    'id': 'f19c57a2-7d89-40f9-a24a-df5a23331e44',
                    'type': 'tool_call'
                }
            ],
            invalid_tool_calls=[],
            usage_metadata={'input_tokens': 298, 'output_tokens': 51, 'total_tokens': 349}
        ),
        HumanMessage(
            content='Please draft and send an email to my professor letting them know I will visit tomorrow.',
            additional_kwargs={},
            response_metadata={},
            id='4924b214-6f2f-48ff-950f-5088d12e7d3e'
        ),
        AIMessage(
            content='',
            additional_kwargs={},
            response_metadata={
                'model': 'llama3.2:3b',
                'created_at': '2026-02-04T08:17:11.7411182Z',
                'done': True,
                'done_reason': 'stop',
                'total_duration': 1661787500,
                'load_duration': 750410300,
                'prompt_eval_count': 

In [19]:
# Interrupt 없이 전송된것을 확인 인코딩이 깨진 것은 모델 성능 탓 
response["messages"][-1].content

'{"name": "send_email_tool", "parameters": {"to": "jtoh@intellicode.co.kr", "subject": "\\u cease \\u e1\\u e4\\u c5 \\u ce8 \\u ad8 \\u ac1 \\u cc3 \\u bc0 \\u ac18", "body": "\\u ce7 \\u ec08 \\u ac19 \\u e25c \\u ac1d \\u ec99 \\u bbf4 \\u bd81 \\u adc9 \\u bbfc \\u b8cc \\u ac00 \\u e28c \\u b974 \\u ad38 \\u b958 \\u ac11 \\u cd92 \\u e25c \\u ac1d \\u ec99 \\u bbf4 \\u bd81 \\u adc9"}}'

---
## PII Detection

In [27]:
@tool
def save_customer_feedback(feedback: str) -> str:
    """고객 피드백을 저장하는 도구"""
    return f"📥 고객 피드백 저장 완료: {feedback}"

In [28]:
from langchain.agents.middleware import LLMToolEmulator, PIIMiddleware

agent = create_agent(
    model,
    tools=[save_customer_feedback],
    middleware=[
        LLMToolEmulator(model=model),
        # 이메일 주소는 전부 마스킹 처리
        PIIMiddleware("email", strategy="redact", apply_to_input=True),


        # 카드번호는 마지막 4자리만 남기고 나머지 마스킹 처리
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True),
    ],
)

In [29]:
prompt = "Hello, I'm jtoh(email : jtoh@intellicode.co.kr) , my credit card number is 1234123443214321"

response = agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]},
    {"configurable": {"thread_id": "PII-a"}}
)

print(response["messages"][0].content) # 가려진 거 확인

Hello, I'm jtoh(email : [REDACTED_EMAIL]) , my credit card number is ************4321


Custom도 가능

In [30]:
phone_number_detector_regex = r"\b(010)[-\s]?(\d{3,4})[-\s]?(\d{4})\b"
api_key_detector_regex = r"sk-\w{32}"

# 커스텀 PII 미들웨어 생성
phone_masking_middleware = PIIMiddleware(
    # pii_type: "phone_number" 라는 커스텀 이름 지정
    pii_type="phone_number",

    # detector: 위에서 만든 정규식 전달
    detector=phone_number_detector_regex,
    # strategy: "mask" (마스킹)
    # 마스킹은 기본적으로 마지막 4자리를 제외하고 마스킹
    strategy="mask",

    # apply_to_input: 사용자 입력에 적용
    apply_to_input=True,

)

# 커스텀 PII 미들웨어 생성
phone_masking_middleware = PIIMiddleware(
    pii_type="api_key",
    detector=api_key_detector_regex,
    strategy="block",
    apply_to_input=True,
)

In [33]:
from langchain.agents.middleware import LLMToolEmulator, PIIMiddleware

agent = create_agent(
    model,
    tools=[save_customer_feedback],
    middleware=[
        LLMToolEmulator(model=model),
        PIIMiddleware(
            pii_type="phone_number",
            detector=phone_number_detector_regex,
            strategy="mask",
            apply_to_input=True,
        )
    ],
)

prompt = "Hello, I'm jtoh(phone number : 01012345678, email : jtoh@intellicode.co.kr) , my credit card number is 1234123443214321"

response = agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]},
    {"configurable": {"thread_id": "PII-a"}}
)

print(response["messages"][0].content) # 가려진 거 확인

Hello, I'm jtoh(phone number : ****5678, email : jtoh@intellicode.co.kr) , my credit card number is 1234123443214321


In [34]:
from langchain.agents.middleware import LLMToolEmulator, PIIMiddleware

agent = create_agent(
    model,
    tools=[save_customer_feedback],
    middleware=[
        LLMToolEmulator(model=model),
        PIIMiddleware(
            pii_type="api_key",
            detector=api_key_detector_regex,
            strategy="block",  # 완전 중단 에러 유발 
            apply_to_input=True,
        )
    ],
)

prompt = "Hello, I'm jtoh(api key : sk-12345678901234567890123456789012) , my credit card number is 1234123443214321"

response = agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]},
    {"configurable": {"thread_id": "PII-a"}}
)

rprint(response)
print(response["messages"][0].content) # 가려진 거 확인

PIIDetectionError: Detected 1 instance(s) of api_key in text content

In [ ]:
## Summarization 도 가능
# model , tirgger(발동조건), keep(몇개 저장), summary_prompt(요약 프롬프트), trim_token_to_summarize(요약 토큰 제한)
tools = []

agent = create_agent(
    model,
    tools,
    middleware=[
        SummarizationMiddleware(
            model=model,
            trigger={"messages" : 5}
            keep={"messages" : 10},
            trim_token_to_summarize=4000,

        )
    ],
    checkpointer=InMemorySaver(),
)